In [4]:
import pandas as pd

df = pd.read_csv("D:\student_performance4_raw3_25cols.csv")

key_duplicates_count = df.duplicated(subset = ['student_id', 'subject_name', 'month']).sum()

print("Number of duplicated records", key_duplicates_count)

key_duplicates_rows = df[df.duplicated(subset = ['student_id', 'subject_name', 'month'], keep = False)]

print("duplicated rows: ")
print(key_duplicates_rows)

Number of duplicated records 0
duplicated rows: 
Empty DataFrame
Columns: [student_id, name, age, gender, grade_level, subject_name, teacher_name, difficulty_level, hours_per_week, description, month, score, sleep_hours, study_hours, attendance_rate, internet_access, free_time_activity, parent_education, homework_completion_rate, extracurricular_activities, family_size, previous_gpa, school_transport, health_condition, tutoring]
Index: []

[0 rows x 25 columns]


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\pc\AppData\Local\Temp\ipykernel_3756\329134982.py:3: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv("D:\student_performance4_raw3_25cols.csv")


In [6]:
print("Shape", df.shape)

Shape (1000, 25)


In [9]:
print("Columns", df.columns)

Columns Index(['student_id', 'name', 'age', 'gender', 'grade_level', 'subject_name',
       'teacher_name', 'difficulty_level', 'hours_per_week', 'description',
       'month', 'score', 'sleep_hours', 'study_hours', 'attendance_rate',
       'internet_access', 'free_time_activity', 'parent_education',
       'homework_completion_rate', 'extracurricular_activities', 'family_size',
       'previous_gpa', 'school_transport', 'health_condition', 'tutoring'],
      dtype='object')


In [10]:
print("Columns", df.columns.to_list())

Columns ['student_id', 'name', 'age', 'gender', 'grade_level', 'subject_name', 'teacher_name', 'difficulty_level', 'hours_per_week', 'description', 'month', 'score', 'sleep_hours', 'study_hours', 'attendance_rate', 'internet_access', 'free_time_activity', 'parent_education', 'homework_completion_rate', 'extracurricular_activities', 'family_size', 'previous_gpa', 'school_transport', 'health_condition', 'tutoring']


In [11]:
print(df.head())

   student_id           name  age  gender grade_level subject_name  \
0           1  Alice Johnson   16  Female    Grade 10         Math   
1           1  Alice Johnson   16  Female    Grade 10         Math   
2           1  Alice Johnson   16  Female    Grade 10         Math   
3           1  Alice Johnson   16  Female    Grade 10         Math   
4           1  Alice Johnson   16  Female    Grade 10      Science   

  teacher_name difficulty_level  hours_per_week  \
0    Mr. Ahmed           Medium               5   
1    Mr. Ahmed           Medium               5   
2    Mr. Ahmed           Medium               5   
3    Mr. Ahmed           Medium               5   
4   Ms. Fatima             Hard               4   

                                    description  ... internet_access  \
0        Basic mathematics concepts and algebra  ...             Yes   
1        Basic mathematics concepts and algebra  ...             Yes   
2        Basic mathematics concepts and algebra  ...    

In [12]:
print(df.isnull().sum())

student_id                      0
name                            0
age                             0
gender                          0
grade_level                     0
subject_name                    0
teacher_name                    0
difficulty_level                0
hours_per_week                  0
description                     0
month                           0
score                          50
sleep_hours                    30
study_hours                    36
attendance_rate                 0
internet_access                 0
free_time_activity              0
parent_education              248
homework_completion_rate       44
extracurricular_activities      0
family_size                    22
previous_gpa                   23
school_transport                0
health_condition                0
tutoring                        0
dtype: int64


In [14]:
print(df.duplicated().sum())

0


In [15]:
print(df.duplicated(subset = ['student_id', 'subject_name', 'month']).sum())

0


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("D:\student_performance4_raw3_25cols.csv")

df["subject_name"] = df["subject_name"].astype(str).str.strip().str.lower()

impute_cols = ["score", "homework_completion_rate", "study_hours", 
               "sleep_hours", "previous_gpa", "family_size", "parent_education"]

for c in impute_cols:
    df[c + "_was_imputed"] = False

def safe_mode(s):
    m = s.mode()
    return m.iloc[0] if not m.empty else np.nan

for c in ["score", "homework_completion_rate", "study_hours", "sleep_hours"]:

    mask_before = df[c].isna()

    df[c] = df.groupby(["student_id", "subject_name"])[c].transform(
        lambda g: g.fillna(g.median())
    )

    df[c] = df.groupby("subject_name")[c].transform(
        lambda g: g.fillna(g.median())
    )

    overall_med = df[c].median()

    if not np.isnan(overall_med):
        df[c] = df[c].fillna(overall_med)
    mask_after = df[c].notna()
    df[c + "_was_imputed"] = mask_before & mask_after

for c in ["previous_gpa", "family_size"]:

    mask_before = df[c].isna()

    df[c] = df.groupby("student_id")[c].transform(
        lambda g: g.fillna(g.median())
    )

    overall_med = df[c].median()

    if not np.isnan(overall_med):
        df[c] = df[c].fillna(overall_med)
    mask_after = df[c].notna()
    df[c + "_was_imputed"] = mask_before & mask_after

col = "parent_education"

mask_before = df[col].isna()

student_mode = df.groupby("student_id")[col].agg(lambda s: safe_mode(s.dropna()))

df[col] = df[col].fillna(df["student_id"].map(student_mode))

overall_mode = safe_mode(df[col].dropna())

if pd.notna(overall_mode):
    df[col] = df[col].fillna(overall_mode)
mask_after = df[col].notna()
df[col + "_was_imputed"] = mask_before & mask_after

print("Missing after imputation:")
print(df[impute_cols].isnull().sum())

df.to_csv("student_performance_cleaned.csv", index=False)
print("Saved student_performance_cleaned.csv")

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\pc\AppData\Local\Temp\ipykernel_3756\1767474053.py:4: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv("D:\student_performance4_raw3_25cols.csv")


Missing after imputation:
score                       0
homework_completion_rate    0
study_hours                 0
sleep_hours                 0
previous_gpa                0
family_size                 0
parent_education            0
dtype: int64
Saved student_performance_cleaned.csv


In [25]:
df = pd.read_csv("student_performance_cleaned.csv")

df["month"] = pd.to_datetime(df["month"], errors = "coerce")

def categorize_performance(score):
    if score >= 85:
        return "High"
    elif score >= 70:
        return "Medium"
    else:
        return "Low"

df["performance_category"] = df["score"].apply(categorize_performance)

df.to_csv("student_performance_cleaned.csv", index = False)

print(df[["student_id", "subject_name", "month", "score", "performance_category"]].head(10))

   student_id subject_name      month  score performance_category
0           1         math 2023-09-01   54.0                  Low
1           1         math 2023-10-01   56.0                  Low
2           1         math 2023-11-01   58.0                  Low
3           1         math 2023-12-01   66.0                  Low
4           1      science 2023-09-01   54.0                  Low
5           1      science 2023-10-01   52.0                  Low
6           1      science 2023-11-01   54.0                  Low
7           1      science 2023-12-01   58.0                  Low
8           1      english 2023-09-01   56.0                  Low
9           1      english 2023-10-01   71.0               Medium


In [26]:
print(df.isnull().sum())

student_id                              0
name                                    0
age                                     0
gender                                  0
grade_level                             0
subject_name                            0
teacher_name                            0
difficulty_level                        0
hours_per_week                          0
description                             0
month                                   0
score                                   0
sleep_hours                             0
study_hours                             0
attendance_rate                         0
internet_access                         0
free_time_activity                      0
parent_education                        0
homework_completion_rate                0
extracurricular_activities              0
family_size                             0
previous_gpa                            0
school_transport                        0
health_condition                  

In [27]:
print(df.head())

   student_id           name  age  gender grade_level subject_name  \
0           1  Alice Johnson   16  Female    Grade 10         math   
1           1  Alice Johnson   16  Female    Grade 10         math   
2           1  Alice Johnson   16  Female    Grade 10         math   
3           1  Alice Johnson   16  Female    Grade 10         math   
4           1  Alice Johnson   16  Female    Grade 10      science   

  teacher_name difficulty_level  hours_per_week  \
0    Mr. Ahmed           Medium               5   
1    Mr. Ahmed           Medium               5   
2    Mr. Ahmed           Medium               5   
3    Mr. Ahmed           Medium               5   
4   Ms. Fatima             Hard               4   

                                    description  ... health_condition  \
0        Basic mathematics concepts and algebra  ...           Normal   
1        Basic mathematics concepts and algebra  ...           Normal   
2        Basic mathematics concepts and algebra  ... 